In [1]:
import casadi as ca
import control
import numpy as np
import cvxpy as cp

In [2]:
m = 1
c = 1
k = 1

A = np.array([
    [0, 1],
    [-k/m, -c/m]
])

B = np.array([
    [0],
    [1/m]
])

G = np.eye(2)
QN = (1e-1)**2*np.eye(2)  # process noise
RN = (1e-2)**2*np.eye(1)  # measurement noise
Q = np.eye(2)
R = np.eye(1)

C = np.array([[1, 0]])

L, _, _ = control.lqe(A, G, C, QN, RN)
K, _, _ = control.lqr(A, B, Q, R)

c_w = 1  # bound on processs noise 
c_v = 1  # bound on measurement noise

In [3]:
print('A', A)
print('L', L)
print('K', K)

A [[ 0.  1.]
 [-1. -1.]]
L [[10.28516255]
 [ 2.89228433]]
K [[0.41421356 0.68179283]]


What is largest possible magnitude of Lv + w

In [4]:
## upper bound on estimator disturbance
c_e = np.linalg.svd(L).S[0]*c_v + c_w
c_e

11.684094594472151

In [5]:
def slemma_prob(A, B, alpha=0, lam_guess=1, delta_lam=0.1, delta_tol=0.001):
    n = A.shape[0]
    lam = lam_guess
    count = 0
    while True:
        n = 2
        P = cp.Variable((n, n), 'P', symmetric=True)
        mu = cp.Variable((1, 1), 'mu')
        constraints = [
            P >> np.eye(n),  # P > 0, P > I same due to homogeneity property
            mu >> 0,
        ]
        alpha = 0
        constraints += [
            cp.bmat([
                [A.T*P + P*A + lam*P + alpha*P, P*B],
                [B.T*P, -lam*mu*np.eye(2)]
            ]) << 0
        ]

        dir_search = 1

        prob = cp.Problem(objective=cp.Minimize(mu), constraints=constraints)
        res = prob.solve(solver='SCS', verbose=False, max_iters=100000)
        #print(prob.status)
        if prob.status == 'infeasible':
            delta_lam /= 2
            lam -= dir_search*delta_lam
            if count == 0:
                print('initial guess must be feasible')
                break
        elif prob.status == 'optimal' or  prob.status == 'optimal_inaccurate':
            #print('ellipse size', np.sqrt(np.linalg.svd(P.value/mu.value).S))
            if delta_lam < delta_tol:
                break
            else: 
                delta_lam *= 2
                lam += dir_search*delta_lam
        elif prob.status == 'infeasible_inaccurate':
            delta_lam /= 2
            lam -= dir_search*delta_lam
        else:
            print('unknown status', prob.status)
            break
        #print('lam', lam, 'delta_lam', delta_lam)
        count += 1
    output_scaling = np.sqrt(np.linalg.svd(P.value/mu.value).S)[0]
    return output_scaling

In [6]:
e_bibo_scaling = slemma_prob(A=A+L*C, B=np.eye(2), lam_guess=0.01)
e_bound = e_bibo_scaling*c_e
e_bound

5.8277607763271995

In [7]:
## upper bound on control disturbance
c_k = np.linalg.svd(B*K).S[0]*e_bound + c_w
c_k

5.649130042699066

In [8]:
x_bibo_scaling = slemma_prob(A=A+B*K, B=np.eye(2), lam_guess=0.01)
x_bound = x_bibo_scaling*c_k
x_bound

0.06861795557974419

$$ V_0 = \mu || w||_2^2 $$

$$ V(x) = x^T P x < \mu || w||_2^2 $$

$$ V(x) = x^T (P/ \mu) x < || w||_2^2 $$

$$ V(x) = x^T (P/ (\mu ||w||_2^2)) x = 1$$
bound of invariant set